In [ ]:
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
import fasttext
import umap
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

tokens_by_class = {
    'TYPE': set(),
    'BRAND': set(),
    'VOLUME': set(),
    'PERCENT': set()
}

In [ ]:
# Настройки для визуализаций
sns.set_theme(style="whitegrid", palette="viridis")
plt.rcParams['figure.figsize'] = (12, 7)
warnings.filterwarnings('ignore')

# Константы для путей к файлам
DATA_RAW_PATH = '../../data/raw/'
TRAIN_FILE = DATA_RAW_PATH + 'train.csv'
SUBMISSION_FILE = DATA_RAW_PATH + 'submission.csv'

In [ ]:
df_train = pd.read_csv(TRAIN_FILE, sep=';')
df_submission = pd.read_csv(SUBMISSION_FILE, sep=';')

print("Обучающий набор данных (train.csv):")
df_train.info()
print("\n")

print("\n" + "=" * 50 + "\n")

print("Тестовый набор данных (submission.csv):")
df_submission.info()
print("\n")

In [ ]:
print("Тип данных колонки 'annotation' до преобразования:", df_train['annotation'].dtype)

# Применяем безопасный парсинг
df_train['annotation'] = df_train['annotation'].apply(ast.literal_eval)

print("Тип данных колонки 'annotation' после преобразования:", df_train['annotation'].dtype)
print("\nПример данных в колонке 'annotation' после преобразования:")
print(df_train['annotation'].iloc[5])

Анализ длины запросов

In [ ]:
df_train['query_len_char'] = df_train['sample'].str.len()
df_train['query_len_token'] = df_train['sample'].str.split().str.len()

df_submission['query_len_char'] = df_submission['sample'].str.len()
df_submission['query_len_token'] = df_submission['sample'].str.split().str.len()

print("Статистика по длинам запросов в обучающем наборе:")
display(df_train[['query_len_char', 'query_len_token']].describe())

print("\nСтатистика по длинам запросов в тестовом наборе:")
display(df_submission[['query_len_char', 'query_len_token']].describe())


plt.figure(figsize=(14, 6))
sns.histplot(df_train['query_len_char'], color='blue', label='Train', kde=True, stat="density", linewidth=0)
sns.histplot(df_submission['query_len_char'], color='red', label='Submission', kde=True, stat="density", linewidth=0, alpha=0.6)
plt.title('Распределение длин запросов в символах (Train vs Submission)')
plt.xlabel('Длина запроса в символах')
plt.ylabel('Плотность')
plt.legend()
plt.show()

plt.figure(figsize=(14, 6))
sns.histplot(df_train['query_len_token'], color='blue', label='Train', kde=True, stat="density", linewidth=0, binwidth=1)
sns.histplot(df_submission['query_len_token'], color='red', label='Submission', kde=True, stat="density", linewidth=0, alpha=0.6, binwidth=1)
plt.title('Распределение длин запросов в токенах (Train vs Submission)')
plt.xlabel('Длина запроса в токенах')
plt.ylabel('Плотность')
plt.legend()
plt.xlim(0, 20)
plt.xticks(range(0, 21))
plt.show()

Анализ мультиязычности

In [ ]:
def contains_latin(text):
    if not isinstance(text, str):
        return False
    return any('a' <= char.lower() <= 'z' for char in text)

In [ ]:
df_train['contains_latin'] = df_train['sample'].apply(contains_latin)
df_submission['contains_latin'] = df_submission['sample'].apply(contains_latin)

train_latin_ratio = df_train['contains_latin'].mean()
submission_latin_ratio = df_submission['contains_latin'].mean()

print(f"Доля запросов с латиницей в обучающем наборе: {train_latin_ratio:.2%}")
print(f"Доля запросов с латиницей в тестовом наборе: {submission_latin_ratio:.2%}")

print("\nПримеры запросов с латиницей из обучающего набора:")
display(df_train[df_train['contains_latin']].sample(10, random_state=42))

Анализ баланса классов сущностей

In [ ]:
def get_entity_type(tag):
    if '-' in tag:
        return tag.split('-')[1]
    return tag

In [ ]:
entity_counter = Counter()
for annotation_list in df_train['annotation']:
    for _, _, tag in annotation_list:
        entity_type = get_entity_type(tag)
        if entity_type != 'O':
            entity_counter[entity_type] += 1

df_entity_counts = pd.DataFrame(entity_counter.items(), columns=['Entity', 'Count']).sort_values('Count', ascending=False)

print("Распределение сущностей в обучающем наборе:")
display(df_entity_counts)


plt.figure(figsize=(10, 6))
sns.barplot(x='Entity', y='Count', data=df_entity_counts)
plt.title('Распределение количества сущностей по типам')
plt.xlabel('Тип сущности')
plt.ylabel('Количество')
for index, row in df_entity_counts.iterrows():
    plt.text(row.name, row.Count, row.Count, color='black', ha="center", va="bottom")
plt.show()

Частотный анализ

In [ ]:
all_tokens = df_train['sample'].dropna().str.lower().str.split().sum()

token_counts = Counter(all_tokens)

print("Топ-30 самых частотных слов в обучающем наборе:")
display(pd.DataFrame(token_counts.most_common(30), columns=['Токен', 'Частота']))

Анализ неоднозначности токенов

In [ ]:
token_to_tags = {}

for _, row in df_train.iterrows():
    query = row['sample']
    annotations = row['annotation']
    
    for start, end, tag in annotations:
        token = query[start:end].lower()
        entity_type = get_entity_type(tag) 
        
        if token not in token_to_tags:
            token_to_tags[token] = set()
        token_to_tags[token].add(entity_type)

ambiguous_tokens = {token: tags for token, tags in token_to_tags.items() if len(tags) > 1}

df_ambiguous = pd.DataFrame(ambiguous_tokens.items(), columns=['Токен', 'Возможные теги'])

print(f"Найдено {len(df_ambiguous)} неоднозначных токенов (могут иметь разные теги в разных контекстах).")
print("\nПримеры 20 самых частотных неоднозначных токенов:")

df_ambiguous['Частота'] = df_ambiguous['Токен'].apply(lambda x: token_counts.get(x, 0))
df_ambiguous = df_ambiguous.sort_values('Частота', ascending=False)

display(df_ambiguous.head(20).reset_index(drop=True))

Анализ контекстного окружения сущностей

In [ ]:
context_before = {entity: Counter() for entity in ['TYPE', 'BRAND', 'VOLUME', 'PERCENT']}
context_after = {entity: Counter() for entity in ['TYPE', 'BRAND', 'VOLUME', 'PERCENT']}

for _, row in df_train.iterrows():
    query_tokens = str(row['sample']).lower().split()
    
    current_pos = 0
    token_tags = ['O'] * len(query_tokens)
    token_indices = []
    for token in query_tokens:
        start = row['sample'].lower().find(token, current_pos)
        end = start + len(token)
        token_indices.append((start, end))
        current_pos = end

    for start_ann, end_ann, tag_ann in row['annotation']:
        entity_type = get_entity_type(tag_ann)
        if entity_type == 'O':
            continue
            
        for i, (start_tok, end_tok) in enumerate(token_indices):
            if max(start_ann, start_tok) < min(end_ann, end_tok):
                
                if i > 0:
                    prev_token = query_tokens[i-1]
                    context_before[entity_type][prev_token] += 1
                
                if i < len(query_tokens) - 1:
                    next_token = query_tokens[i+1]
                    context_after[entity_type][next_token] += 1

for entity_type in ['TYPE', 'BRAND', 'VOLUME', 'PERCENT']:
    print(f"\n===== Контекст для сущности: {entity_type} =====")
    
    df_before = pd.DataFrame(context_before[entity_type].most_common(10), columns=['Слово до', 'Частота'])
    df_after = pd.DataFrame(context_after[entity_type].most_common(10), columns=['Слово после', 'Частота'])
    
    print("Топ-10 слов, встречающихся ДО сущности:")
    display(df_before)
    
    print("\nТоп-10 слов, встречающихся ПОСЛЕ сущности:")
    display(df_after)

Анализ пространства признаков

In [ ]:
for _, row in df_train.iterrows():
    query = row['sample']
    annotations = row['annotation']
    
    for start, end, tag in annotations:
        entity_type = get_entity_type(tag)
        if entity_type in tokens_by_class:
            token = query[start:end].lower()
            tokens_by_class[entity_type].add(token)

all_unique_tokens = []
labels = []
for entity_type, token_set in tokens_by_class.items():
    tokens_list = list(token_set)
    all_unique_tokens.extend(tokens_list)
    labels.extend([entity_type] * len(tokens_list))

print("Подготовка данных завершена.")
for entity_type, token_set in tokens_by_class.items():
    print(f"Найдено {len(token_set)} уникальных токенов для класса '{entity_type}'")

In [ ]:
print("Загрузка модели FastText... Это может занять несколько минут.")
ft_model_path = '../../models/external/cc.ru.300.bin'  # TODO Укажите свой путь к модели
ft_model = fasttext.load_model(ft_model_path)
print("Модель FastText успешно загружена.")

ft_embeddings = np.array([ft_model.get_word_vector(token) for token in all_unique_tokens])

print("Применение UMAP для FastText эмбеддингов...")
reducer_ft = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)
embedding_2d_ft = reducer_ft.fit_transform(ft_embeddings)

plt.figure(figsize=(12, 10))
df_plot_ft = pd.DataFrame({'x': embedding_2d_ft[:, 0], 'y': embedding_2d_ft[:, 1], 'label': labels})
sns.scatterplot(data=df_plot_ft, x='x', y='y', hue='label', s=20, alpha=0.7)
plt.title('Визуализация эмбеддингов токенов (FastText + UMAP)')
plt.xlabel('UMAP компонента 1')
plt.ylabel('UMAP компонента 2')
plt.legend(title='Тип сущности')
plt.show()

Векторизация TF-IDF и n-gramms

In [ ]:
print("Векторизация с помощью TF-IDF на n-граммах символов...")
tfidf_vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 4))
tfidf_embeddings = tfidf_vectorizer.fit_transform(all_unique_tokens)

tfidf_embeddings_normalized = normalize(tfidf_embeddings)


print("Применение UMAP для TF-IDF эмбеддингов...")
reducer_tfidf = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42, metric='hellinger')
embedding_2d_tfidf = reducer_tfidf.fit_transform(tfidf_embeddings_normalized)

plt.figure(figsize=(12, 10))
df_plot_tfidf = pd.DataFrame({'x': embedding_2d_tfidf[:, 0], 'y': embedding_2d_tfidf[:, 1], 'label': labels})
sns.scatterplot(data=df_plot_tfidf, x='x', y='y', hue='label', s=20, alpha=0.7)
plt.title('Визуализация токенов (TF-IDF на n-граммах символов + UMAP)')
plt.xlabel('UMAP компонента 1')
plt.ylabel('UMAP компонента 2')
plt.legend(title='Тип сущности')
plt.show()